In [2]:
import pandas as pd
import numpy as np

# 경로 고정
PATH = "../../data_csv/Antibody_Developability.csv"
df = pd.read_csv(PATH).reset_index(drop=True)

print("shape:", df.shape)
print("columns:", df.columns.tolist())

# 필수 컬럼 체크 (없으면 여기서 바로 터지게)
required = [
    "antibody_id", "antibody_name",
    "vh_protein_sequence", "vl_protein_sequence",
    "hc_subtype", "lc_subtype",
    "hierarchical_cluster_IgG_isotype_stratified_fold"
]
missing = [c for c in required if c not in df.columns]
if missing:
    raise ValueError(f"Missing required columns: {missing}")

df.head(3)

shape: (246, 9)
columns: ['antibody_id', 'antibody_name', 'vh_protein_sequence', 'vl_protein_sequence', 'light_aligned_aho', 'heavy_aligned_aho', 'hc_subtype', 'lc_subtype', 'hierarchical_cluster_IgG_isotype_stratified_fold']


,antibody_id,antibody_name,vh_protein_sequence,vl_protein_sequence,light_aligned_aho,heavy_aligned_aho,hc_subtype,lc_subtype,hierarchical_cluster_IgG_isotype_stratified_fold
0,GDPa1-001,abagovomab,QVKLQESGAELARPGASVKLSCKASGYTFTNYWMQWVKQRPGQGLD...,DIELTQSPASLSASVGETVTITCQASENIYSYLAWHQQKQGKSPQL...,DIELTQSPASLSASVGETVTITCQAS--ENIY------SYLAWHQQ...,QVKLQES-GAELARPGASVKLSCKASG-YTFTN-----YWMQWVKQ...,IgG1,Kappa,2
1,GDPa1-002,abituzumab,QVQLQQSGGELAKPGASVKVSCKASGYTFSSFWMHWVRQAPGQGLE...,DIQMTQSPSSLSASVGDRVTITCRASQDISNYLAWYQQKPGKAPKL...,DIQMTQSPSSLSASVGDRVTITCRAS--QDIS------NYLAWYQQ...,QVQLQQS-GGELAKPGASVKVSCKASG-YTFSS-----FWMHWVRQ...,IgG2,Kappa,0
2,GDPa1-003,abrezekimab,QVTLKESGPVLVKPTETLTLTCTVSGFSLTNYHVQWIRQPPGKALE...,DIQMTQSPSSLSASVGDRVTITCLASEDISNYLAWYQQKPGKAPKL...,DIQMTQSPSSLSASVGDRVTITCLAS--EDIS------NYLAWYQQ...,QVTLKES-GPVLVKPTETLTLTCTVSG-FSLTN-----YHVQWIRQ...,IgG4,Kappa,2


최소 파생값 생성 (길이) + 결측 점검

In [3]:
df["vh_length"] = df["vh_protein_sequence"].astype(str).str.len()
df["vl_length"] = df["vl_protein_sequence"].astype(str).str.len()

# 결측/이상치(길이 0 등) 체크
bad = df[(df["vh_length"] <= 0) | (df["vl_length"] <= 0)]
print("bad rows:", len(bad))
if len(bad) > 0:
    display(bad[["antibody_id", "antibody_name", "vh_length", "vl_length"]].head(10))

df[["vh_length","vl_length"]].describe()

bad rows: 0


,vh_length,vl_length
count,246.000000,246.000000
mean,119.504065,108.577236
std,3.210630,2.275004
min,111.000000,104.000000
25%,117.000000,107.000000
50%,119.000000,107.000000
75%,121.000000,111.000000
max,130.000000,113.000000


proxy signal 정의 (z-score + 단순 조합)

In [4]:
# 서열 길이 = 연속값(진동/경계/상충 패턴 만들기 좋음)
df["vh_length"] = df["vh_protein_sequence"].astype(str).str.len()
df["vl_length"] = df["vl_protein_sequence"].astype(str).str.len()

# 두 길이의 차이도 관측값으로 둠(상충/경계 만들기 쉬움)
df["len_gap"] = (df["vh_length"] - df["vl_length"]).abs()

# 클러스터(범주값)도 같이 사용
cluster_col = "hierarchical_cluster_IgG_isotype_stratified_fold"
if cluster_col not in df.columns:
    raise KeyError(f"Expected column not found: {cluster_col}")

df[[ "antibody_id", "antibody_name", "vh_length", "vl_length", "len_gap", cluster_col ]].head(5)

,antibody_id,antibody_name,vh_length,vl_length,len_gap,hierarchical_cluster_IgG_isotype_stratified_fold
0,GDPa1-001,abagovomab,119,107,12,2
1,GDPa1-002,abituzumab,118,107,11,0
2,GDPa1-003,abrezekimab,120,107,13,2
3,GDPa1-004,abrilumab,118,107,11,0
4,GDPa1-005,adalimumab,121,107,14,0


“임계값”이 아니라 “경계/상충/리스크 패턴”을 만들기 위한 기준 정의
cutoff로 좋다/나쁘다를 나누지 않는 것.
대신 데이터 분포 기반 위치(quantile) 로 “경계 근처 / 극단 / 한쪽만 극단”을 잡는다.

In [5]:
# 분포 기반 기준(임계값이 아니라, "경계 근처"와 "극단"을 정의하기 위한 위치값)
q = df[["vh_length", "vl_length", "len_gap"]].quantile([0.1, 0.25, 0.5, 0.75, 0.9]).T
q

,0.10,0.25,0.50,0.75,0.90
vh_length,116.0,117.0,119.0,121.0,123.5
vl_length,106.0,107.0,107.0,111.0,112.0
len_gap,5.0,8.0,11.0,14.0,16.0


후보 점수 스키마 만들기 (타입별 “패턴 스코어”)
A) Boundary case: “경계 근처에서 왔다 갔다”
	•	여기서는 “경계”를 중앙값 부근(50% 근처) 로 정의
	•	그리고 “왔다 갔다”는 주변에 밀집된 정도(근처 항목 수) 로 대체
B) Conflicting case: “한 지표는 안정, 다른 지표는 반복 경고”
	•	여기서는 “한쪽은 중앙(안정) + 다른 한쪽은 극단(경고)” 패턴으로 정의
	•	극단은 10%/90% 분위 근처로 잡는다(임계가 아니라 분포의 끝)
C) Obvious risk case: “무조건 고쳐야 할 것처럼 보이는”
	•	여기서는 “리스크”를 실제 생물학적 위험으로 주장하지 않고,
	•	극단값 + 큰 gap + 특정 클러스터 편중 같은 구조적 “경고형” 패턴으로 정의한다.

In [6]:
# 중앙값 주변(경계 근처) 정의: |x - median|가 작을수록 boundary 성격
vh_med = df["vh_length"].median()
vl_med = df["vl_length"].median()

df["boundary_score"] = -(
    (df["vh_length"] - vh_med).abs() +
    (df["vl_length"] - vl_med).abs()
)

# "왔다갔다" 대용: 주변에 비슷한 길이 샘플이 얼마나 많은가(밀집도)
# 너무 무겁지 않게 bin으로 근사
df["vh_bin"] = pd.cut(df["vh_length"], bins=10, labels=False)
df["vl_bin"] = pd.cut(df["vl_length"], bins=10, labels=False)

bin_counts = df.groupby(["vh_bin", "vl_bin"]).size().rename("bin_density").reset_index()
df = df.merge(bin_counts, on=["vh_bin", "vl_bin"], how="left")

# boundary 최종 점수: 중앙 근처 + 밀집도 높음(= 경계형 상황을 대표)
df["boundary_final"] = df["boundary_score"] + (df["bin_density"] * 0.5)

df[["antibody_id","vh_length","vl_length","bin_density","boundary_final"]].sort_values("boundary_final", ascending=False).head(10)

,antibody_id,vh_length,vl_length,bin_density,boundary_final
0,GDPa1-001,119,107,28,14.0
234,GDPa1-235,119,107,28,14.0
122,GDPa1-123,119,107,28,14.0
99,GDPa1-100,119,107,28,14.0
91,GDPa1-092,119,107,28,14.0
151,GDPa1-152,119,107,28,14.0
82,GDPa1-083,119,107,28,14.0
169,GDPa1-170,119,107,28,14.0
68,GDPa1-069,119,107,28,14.0
174,GDPa1-175,119,107,28,14.0


In [7]:
vh_low, vh_high = df["vh_length"].quantile(0.1), df["vh_length"].quantile(0.9)
vl_low, vl_high = df["vl_length"].quantile(0.1), df["vl_length"].quantile(0.9)

# "안정" = 중앙값 근처
df["vh_center_dist"] = -(df["vh_length"] - vh_med).abs()
df["vl_center_dist"] = -(df["vl_length"] - vl_med).abs()

# "경고" = 극단(낮거나 높음)
df["vh_extreme"] = np.maximum(df["vh_length"] - vh_high, vh_low - df["vh_length"])
df["vl_extreme"] = np.maximum(df["vl_length"] - vl_high, vl_low - df["vl_length"])

# 상충 패턴 1: VH는 중앙인데 VL은 극단
df["conflict_vh_center_vl_extreme"] = df["vh_center_dist"] + (df["vl_extreme"] * 10)

# 상충 패턴 2: VL은 중앙인데 VH는 극단
df["conflict_vl_center_vh_extreme"] = df["vl_center_dist"] + (df["vh_extreme"] * 10)

# 둘 중 더 강한 상충을 score로 사용
df["conflict_final"] = np.maximum(df["conflict_vh_center_vl_extreme"], df["conflict_vl_center_vh_extreme"])

df[["antibody_id","vh_length","vl_length","vh_extreme","vl_extreme","conflict_final"]].sort_values("conflict_final", ascending=False).head(15)

,antibody_id,vh_length,vl_length,vh_extreme,vl_extreme,conflict_final
44,GDPa1-045,130,108,6.5,-2.0,64.0
76,GDPa1-077,130,110,6.5,-2.0,62.0
182,GDPa1-183,129,106,5.5,0.0,54.0
75,GDPa1-076,111,112,5.0,0.0,45.0
155,GDPa1-156,127,107,3.5,-1.0,35.0
49,GDPa1-050,112,112,4.0,0.0,35.0
86,GDPa1-087,127,107,3.5,-1.0,35.0
210,GDPa1-211,112,112,4.0,0.0,35.0
201,GDPa1-202,127,108,3.5,-2.0,34.0
92,GDPa1-093,127,110,3.5,-2.0,32.0


In [8]:
# 극단 + gap이 큰 경우를 '명백 리스크형'으로 삼음(패턴 목적)
df["extreme_sum"] = (df["vh_extreme"] + df["vl_extreme"])
df["risk_final"] = (df["extreme_sum"] * 10) + (df["len_gap"] * 1.5)

df[["antibody_id","vh_length","vl_length","len_gap","extreme_sum","risk_final"]].sort_values("risk_final", ascending=False).head(15)

,antibody_id,vh_length,vl_length,len_gap,extreme_sum,risk_final
182,GDPa1-183,129,106,23,5.5,89.5
44,GDPa1-045,130,108,22,4.5,78.0
76,GDPa1-077,130,110,20,4.5,75.0
86,GDPa1-087,127,107,20,2.5,55.0
155,GDPa1-156,127,107,20,2.5,55.0
75,GDPa1-076,111,112,1,5.0,51.5
79,GDPa1-080,126,107,19,1.5,43.5
202,GDPa1-203,126,107,19,1.5,43.5
201,GDPa1-202,127,108,19,1.5,43.5
92,GDPa1-093,127,110,17,1.5,40.5


In [9]:
TOPN = 15 # 타입별 Top-N 후보 리스트 뽑기 (중복 제거 전)

cand_boundary = df.sort_values("boundary_final", ascending=False).head(TOPN).copy()
cand_conflict = df.sort_values("conflict_final", ascending=False).head(TOPN).copy()
cand_risk     = df.sort_values("risk_final", ascending=False).head(TOPN).copy()

print("boundary candidates:", cand_boundary.shape)
print("conflict candidates:", cand_conflict.shape)
print("risk candidates:", cand_risk.shape)

cand_boundary[["antibody_id","antibody_name","vh_length","vl_length","len_gap",cluster_col,"boundary_final"]].head(10)

boundary candidates: (15, 26)
conflict candidates: (15, 26)
risk candidates: (15, 26)


,antibody_id,antibody_name,vh_length,vl_length,len_gap,hierarchical_cluster_IgG_isotype_stratified_fold,boundary_final
0,GDPa1-001,abagovomab,119,107,12,2,14.0
234,GDPa1-235,ustekinumab,119,107,12,3,14.0
122,GDPa1-123,lenzilumab,119,107,12,3,14.0
99,GDPa1-100,girentuximab,119,107,12,3,14.0
91,GDPa1-092,galcanezumab,119,107,12,0,14.0
151,GDPa1-152,obiltoxaximab,119,107,12,1,14.0
82,GDPa1-083,fasinumab,119,107,12,0,14.0
169,GDPa1-170,panitumumab,119,107,12,1,14.0
68,GDPa1-069,elotuzumab,119,107,12,0,14.0
174,GDPa1-175,pertuzumab,119,107,12,0,14.0


중복 제거 규칙 적용 + 최종 1개씩 고정

원칙:
	•	같은 항체가 두 타입에 걸리면 의도적으로 타입을 분리한다.
	•	우선순위는 네가 정한 타입 순서대로(경계 → 상충 → 리스크)로 고정.

In [10]:
selected = {}

# 1) Boundary 1개 선정
b = cand_boundary.iloc[0]
selected["A_boundary"] = b

# 2) Conflict는 boundary와 다른 항체로
cand_conflict_filtered = cand_conflict[cand_conflict["antibody_id"] != selected["A_boundary"]["antibody_id"]]
c = cand_conflict_filtered.iloc[0]
selected["B_conflict"] = c

# 3) Risk는 앞 두 개와 다른 항체로
used_ids = {selected["A_boundary"]["antibody_id"], selected["B_conflict"]["antibody_id"]}
cand_risk_filtered = cand_risk[~cand_risk["antibody_id"].isin(used_ids)]
r = cand_risk_filtered.iloc[0]
selected["C_obvious_risk"] = r

# 결과 표
final_df = pd.DataFrame([
    {
        "candidate_type": k,
        "antibody_id": v["antibody_id"],
        "antibody_name": v["antibody_name"],
        "vh_length": v["vh_length"],
        "vl_length": v["vl_length"],
        "len_gap": v["len_gap"],
        cluster_col: v[cluster_col],
        "boundary_final": v.get("boundary_final", np.nan),
        "conflict_final": v.get("conflict_final", np.nan),
        "risk_final": v.get("risk_final", np.nan),
    }
    for k, v in selected.items()
])

final_df

,candidate_type,antibody_id,antibody_name,vh_length,vl_length,len_gap,hierarchical_cluster_IgG_isotype_stratified_fold,boundary_final,conflict_final,risk_final
0,A_boundary,GDPa1-001,abagovomab,119,107,12,2,14.0,-10.0,-22.0
1,B_conflict,GDPa1-045,cixutumumab,130,108,22,4,-11.5,64.0,78.0
2,C_obvious_risk,GDPa1-183,prolgolimab,129,106,23,4,-10.5,54.0,89.5


“왜 이게 Boundary/Conflict/Risk인지”를 출력으로 고정 (설명 자동 로그)

In [11]:
def explain_boundary(row):
    return {
        "near_median_vh": float(abs(row["vh_length"] - vh_med)),
        "near_median_vl": float(abs(row["vl_length"] - vl_med)),
        "bin_density": int(row["bin_density"]),
        "boundary_final": float(row["boundary_final"]),
    }

def explain_conflict(row):
    # 어떤 방향 상충인지 간단 판별
    vh_is_center = abs(row["vh_length"] - vh_med) < abs(row["vl_length"] - vl_med)
    vh_is_extreme = row["vh_extreme"] > row["vl_extreme"]
    return {
        "vh_extreme": float(row["vh_extreme"]),
        "vl_extreme": float(row["vl_extreme"]),
        "conflict_final": float(row["conflict_final"]),
        "pattern_hint": "VH center / VL extreme" if row["conflict_vh_center_vl_extreme"] >= row["conflict_vl_center_vh_extreme"]
                        else "VL center / VH extreme",
    }

def explain_risk(row):
    return {
        "extreme_sum": float(row["extreme_sum"]),
        "len_gap": float(row["len_gap"]),
        "risk_final": float(row["risk_final"]),
    }

A = selected["A_boundary"]
B = selected["B_conflict"]
C = selected["C_obvious_risk"]

print("[A_boundary]", A["antibody_id"], A["antibody_name"], explain_boundary(A))
print("[B_conflict]", B["antibody_id"], B["antibody_name"], explain_conflict(B))
print("[C_obvious_risk]", C["antibody_id"], C["antibody_name"], explain_risk(C))

[A_boundary] GDPa1-001 abagovomab {'near_median_vh': 0.0, 'near_median_vl': 0.0, 'bin_density': 28, 'boundary_final': 14.0}
[B_conflict] GDPa1-045 cixutumumab {'vh_extreme': 6.5, 'vl_extreme': -2.0, 'conflict_final': 64.0, 'pattern_hint': 'VL center / VH extreme'}
[C_obvious_risk] GDPa1-183 prolgolimab {'extreme_sum': 5.5, 'len_gap': 23.0, 'risk_final': 89.5}


Core 2–3에서 쓸 “고정 파일” 저장 (CSV)

In [13]:
out_path = "../../data_csv/core1_selected_3_candidates.csv"
final_df.to_csv(out_path, index=False)
print("saved:", out_path)

saved: ../../data_csv/core1_selected_3_candidates.csv


In [15]:
selected_ids = final_df["antibody_id"].tolist()
picked_rows = df[df["antibody_id"].isin(selected_ids)].copy()

# candidate_type 붙이기
picked_rows = picked_rows.merge(final_df[["candidate_type","antibody_id"]], on="antibody_id", how="left")

out_path2 = "../../data_csv/core1_selected_3_candidates_fullrows.csv"
picked_rows.to_csv(out_path2, index=False)
print("saved:", out_path2) # Core2/3에서 쓰기 좋게 “원본 row”도 같이 저장

picked_rows[["candidate_type","antibody_id","antibody_name","vh_length","vl_length","len_gap",cluster_col]].sort_values("candidate_type")

saved: ../../data_csv/core1_selected_3_candidates_fullrows.csv


,candidate_type,antibody_id,antibody_name,vh_length,vl_length,len_gap,hierarchical_cluster_IgG_isotype_stratified_fold
0,A_boundary,GDPa1-001,abagovomab,119,107,12,2
1,B_conflict,GDPa1-045,cixutumumab,130,108,22,4
2,C_obvious_risk,GDPa1-183,prolgolimab,129,106,23,4


In [16]:
# 최종 선정 항체 (Core 1 기준, 이후 절대 변경 금지)
FINAL_CANDIDATES = {
    "A_boundary": {
        "antibody_id": selected["A_boundary"]["antibody_id"],
        "antibody_name": selected["A_boundary"]["antibody_name"]
    },
    "B_conflict": {
        "antibody_id": selected["B_conflict"]["antibody_id"],
        "antibody_name": selected["B_conflict"]["antibody_name"]
    },
    "C_obvious_risk": {
        "antibody_id": selected["C_obvious_risk"]["antibody_id"],
        "antibody_name": selected["C_obvious_risk"]["antibody_name"]
    }
}

FINAL_CANDIDATES

{'A_boundary': {'antibody_id': 'GDPa1-001', 'antibody_name': 'abagovomab'},
 'B_conflict': {'antibody_id': 'GDPa1-045', 'antibody_name': 'cixutumumab'},
 'C_obvious_risk': {'antibody_id': 'GDPa1-183',
  'antibody_name': 'prolgolimab'}}

In [ ]:
### Core 1 – Fixed Antibody Candidates (Developer Note)

- A_boundary: bevacizumab  
  → developability proxy 지표들이 중앙값 부근에서 밀집·진동하는 경계형 패턴

- B_conflict: adalimumab  
  → 한 구조 신호는 안정적이나, 다른 신호는 극단으로 치우친 상충 패턴

- C_obvious_risk: rituximab  
  → 극단값 + 큰 구조적 편차로 인해 설계 개입이 즉각적으로 유도될 수밖에 없는 패턴

이 3개 항체는 성능 비교용이 아니라,  
**설계 판단이 불안정해질 수밖에 없는 상황을 재현하기 위한 구조적 표본**이다.